In [1]:
import serial
import firebase_admin
from firebase_admin import credentials, db
import json
from datetime import datetime

databaseURL = 'https://sit225-2024-default-rtdb.asia-southeast1.firebasedatabase.app/'
cred_obj = firebase_admin.credentials.Certificate(
    'sit225-2024-firebase-adminsdk-8yopu-1f7163bd72.json'
)
default_app = firebase_admin.initialize_app(cred_obj, {
	'databaseURL':databaseURL
	})


In [ ]:
# Set up serial connection
ser = serial.Serial('/dev/cu.usbmodem2101', 9600, timeout=20)  # Replace 'COM_PORT' with your Arduino port

def upload_to_firebase(data):
    ref = db.reference('sensor_data_2')
    ref.push(data)
#Delete the previous data    
ref = db.reference('sensor_data')

print("Starting data collection...")
while True:
    # Read the data from Arduino
    line = ser.readline().decode('utf-8').strip()
    # Parse the JSON data
    sensor_data = json.loads(line)

    # Record the current timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Prepare the data for Firebase
    data = {
        'timestamp': timestamp,
        'x': sensor_data.get('x'),
        'y': sensor_data.get('y'),
        'z': sensor_data.get('z')
    }

    # Upload data to Firebase
    upload_to_firebase(data)
    print(f"Data uploaded: {data}")

Starting data collection...
Data uploaded: {'timestamp': '2024-08-08 00:22:58', 'x': 0.3052, 'y': 0.6104, 'z': 0.4883}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.4272, 'y': 1.0376, 'z': 1.0986}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.7324, 'y': 1.4038, 'z': 1.4038}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.1221, 'y': 1.6479, 'z': 1.5259}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.3052, 'y': 0.7935, 'z': 0.8545}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.4883, 'y': 0.1831, 'z': 0.061}
Data uploaded: {'timestamp': '2024-08-08 00:22:59', 'x': 0.0, 'y': -0.2441, 'z': -0.4272}
Data uploaded: {'timestamp': '2024-08-08 00:23:00', 'x': 0.1221, 'y': -0.4883, 'z': -0.1831}
Data uploaded: {'timestamp': '2024-08-08 00:23:00', 'x': 0.4883, 'y': 0.0, 'z': 0.4272}
Data uploaded: {'timestamp': '2024-08-08 00:23:00', 'x': 0.061, 'y': 0.7324, 'z': 0.8545}
Data uploaded: {'timestamp': '2024-08-08 00:23:00', 'x': 0.3662, '

In [45]:
import json
import csv

with open('sit225-2024.json') as json_file:
    datas = json.load(json_file)

with open('data_file.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['id','timestamp', 'x', 'y', 'z'])
    # Write the header
    writer.writeheader()
    for id, items in datas.items():
        for id_1, item in items.items():
            item['id'] = id_1  # Add the ID to the data dictionary
            writer.writerow(item)
data_file.close()

In [95]:
import pandas as pd
df = pd.read_csv('data_file.csv')
df

,id,timestamp,x,y,z
0,-O3cCJJCXUTaZkOSasy8,2024-08-07 01:01:05,0.8545,0.3052,0.1221
1,-O3cCJLnLjXRMAnfd9yO,2024-08-07 01:01:05,0.7935,0.2441,0.0610
2,-O3cCJO5HFrafcT7pDLI,2024-08-07 01:01:05,0.6714,0.2441,0.1221
3,-O3cCJQXESJq9fLT5eqH,2024-08-07 01:01:05,0.3662,0.3052,0.4272
4,-O3cCJSuLx1sdLFay6Xs,2024-08-07 01:01:05,0.3052,0.3662,0.4883
...,...,...,...,...,...
21084,-O3coj9ZCTvEr3Ecec9d,2024-08-07 03:53:18,0.3052,0.6714,0.5493
21085,-O3cojBnJnD4TwxTQPtY,2024-08-07 03:53:19,0.3662,0.6714,0.4883
21086,-O3cojDwWmFLzkyc0MZ5,2024-08-07 03:53:19,0.4272,0.5493,0.4272
21087,-O3cojG7yzz8R5KziIRV,2024-08-07 03:53:19,0.4272,0.5493,0.4272


In [96]:
df.dropna(inplace = True)
df['timestamp'] = pd.to_datetime(df.timestamp)
df = df.drop(columns = 'id')
df.set_index('timestamp',inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21089 entries, 2024-08-07 01:01:05 to 2024-08-07 03:53:19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       21089 non-null  float64
 1   y       21089 non-null  float64
 2   z       21089 non-null  float64
dtypes: float64(3)
memory usage: 659.0 KB


In [101]:
#Remove unecessary timestampe
df = df.between_time('01:00:00','01:50:00')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20902 entries, 2024-08-07 01:01:05 to 2024-08-07 01:49:56
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       20902 non-null  float64
 1   y       20902 non-null  float64
 2   z       20902 non-null  float64
dtypes: float64(3)
memory usage: 653.2 KB


In [102]:
def remove_outliers_iqr(df, columns, factor=1.5):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile) for each column
    Q1 = df[columns].quantile(0.25)
    Q3 = df[columns].quantile(0.75)
    
    # Calculate IQR (Interquartile Range)
    IQR = Q3 - Q1
    
    # Define the bounds for outliers
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    
    # Identify outliers
    outliers = (df[columns] < lower_bound) | (df[columns] > upper_bound)
    
    # Filter out outliers
    filtered_df = df[~outliers.any(axis=1)]
    
    return filtered_df

clean_df = remove_outliers_iqr(df, columns=['x', 'y', 'z'])
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11610 entries, 2024-08-07 01:01:05 to 2024-08-07 01:49:56
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       11610 non-null  float64
 1   y       11610 non-null  float64
 2   z       11610 non-null  float64
dtypes: float64(3)
memory usage: 362.8 KB


In [ ]:
import matplotlib.pyplot as plt

df.info()
fig, axs = plt.subplots(2,2, figsize=(16, 9))

#plot there variable in single graph
axs[0,0].plot(clean_df.index, clean_df['x'],linestyle='-')
axs[0,1].plot(clean_df.index, clean_df['y'], label='y')
axs[1,0].plot(clean_df.index, clean_df['z'], label='z')
axs[1,1].plot(clean_df.index, clean_df['x','y','z']